In [ ]:
#@title License
# Copyright 2020 Google LLC.

# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at

# https://www.apache.org/licenses/LICENSE-2.0

# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

How to generate the MathQA-Python dataset?



---



1.   Download the dataset from the MathQA project webpage: https://math-qa.github.io/
2.   Create the mathqa directory in the local colab drive.
3. Unpack the json files (train.json, dev.json, test.json, challenge_test.json) and place them in the mathqa directory.
4. Run the cells below - they will generate the MathQA-Python dataset for the test split. 
5. Repeat the process for other splits if needed.


In [ ]:
!pip install -U git+https://github.com/google/trax.git@220a62303ebf4ad18871aa5607b4dda2f064f2d2

In [ ]:
from trax import data
import json
import numpy as np
import os
import tensorflow as tf

In [ ]:
dataset_path = '/content/mathqa/'

In [ ]:
mathqa_test_gen = data.CreateMathQAInputs(dataset_path=dataset_path, cumulative=False, python_code=True, full_dict=True, train=False, test=True)()
def read_all_problems(mathqa_gen):
  problems = []
  questions = set()
  index = 0
  while True:
    problem = next(mathqa_gen)
    problem_dict = {}
    if problem[0] in questions:
      break
    else:
      problem_dict['text'] = problem[0]
      problem_dict['code'] = problem[1]
      problem_dict['dsl_code'] = problem[2]
      problem_dict['reasoning'] = problem[3].strip('\"').strip("\'")
      problem_dict['answer'] = data.tf_inputs.execute_mathqa_program(problem[0], problem[1].split('\n'))
      problem_dict['task_id'] = index
      np.testing.assert_almost_equal(problem_dict['answer'], data.tf_inputs.execute_mathqa_dsl_program(problem[0], [problem[2]]))
      problems.append(problem_dict)
      questions.add(problem[0])
      index += 1
  return problems

In [ ]:
test_problems = read_all_problems(mathqa_test_gen)

In [ ]:
len(test_problems)

In [ ]:
test_problems[0]